In [ ]:
#install tensorflow-gpu version 1.12.0. Make sure you run Python 3.6 with Cuda 9.0

!pip install tensorflow-gpu == 1.12.0

In [ ]:
###code adapted from the github page: https://github.com/jliphard/DeepEvolve/blob/master/main.py

In [ ]:
#import packages

from __future__ import print_function

from evolver import Evolver

from tqdm import tqdm

import logging

import sys

In [ ]:
#setup logging

logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
)

In [ ]:
def train_genomes(genomes, dataset):
    """Train each genome.

    Args:
        genomes (list): Current population of genomes
        dataset (str): Dataset to use for training/evaluating

    """
    logging.info("***train_genomes(genomes, dataset)***")

    pbar = tqdm(total=len(genomes))

    for genome in genomes:
        genome.train(dataset)
        pbar.update(1)
    
    pbar.close()

In [ ]:
def get_average_accuracy(genomes):
    """Get the average accuracy for a group of genomes.

    Args:
        genomes (list): List of genomes

    Returns:
        float: The average accuracy of a population of genomes.

    """
    total_accuracy = 0

    for genome in genomes:
        total_accuracy += genome.accuracy

    return total_accuracy / len(genomes)

In [ ]:
def generate(generations, population, all_possible_genes, dataset):
    """Generate a network with the genetic algorithm.

    Args:
        generations (int): Number of times to evolve the population
        population (int): Number of networks in each generation
        all_possible_genes (dict): Hyperparameter choices for networks
        dataset (str): Dataset to use for training/evaluating

    """
    logging.info("***generate(generations, population, all_possible_genes, dataset)***")
    
    evolver = Evolver(all_possible_genes)
    
    genomes = evolver.create_population(population)

    #evolve the generation
    for i in range(generations):

        logging.info("***Now in generation %d of %d***" % (i + 1, generations))

        print_genomes(genomes)
        
        #train and get accuracy for genomes
        train_genomes(genomes, dataset)

        #get the average accuracy for this generation
        average_accuracy = get_average_accuracy(genomes)

        #print out the average accuracy each generation
        logging.info("Generation average: %.2f%%" % (average_accuracy * 100))
        logging.info('-'*80) #-----------

        #evolve, except on the last iteration
        if i != generations - 1:
            genomes = evolver.evolve(genomes)

    #sort our final population according to performance
    genomes = sorted(genomes, key=lambda x: x.accuracy, reverse=True)

    #print out the top 5 genomes
    print_genomes(genomes[:5])

In [ ]:
def print_genomes(genomes):
    """Print a list of genomes.

    Args:
        genomes (list): The population of genomes

    """
    logging.info('-'*80)

    for genome in genomes:
        genome.print_genome()

In [ ]:
def main():
    population = 20 #number of genomes in each generation
    
    dataset = 'malaria'

    print("***Dataset:", dataset)
    
    generations = 10 #number of times to evolve the population
    
    all_possible_genes = {
        'dense': [256, 512], 
        'conv_dropout': [0.25, 0.5],
        'dropout': [0.25, 0.5],
        'activation': ['relu', 'elu', 'tanh', 'sigmoid', 'hard_sigmoid', 'softplus', 'linear'],
        'optimizer':  ['rmsprop', 'adam', 'sgd', 'adagrad', 'adadelta', 'adamax', 'nadam']
    }
            
    print("***Evolving for %d generations with population size = %d***" % (generations, population))

    generate(generations, population, all_possible_genes, dataset)

if __name__ == '__main__':
    main()